In [11]:
from plotly.offline import init_notebook_mode, iplot_mpl, download_plotlyjs, plot, iplot

%matplotlib inline
import numpy as np
import plotly_express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True)
from sklearn.preprocessing import LabelEncoder
from statsmodels.compat import lzip
import statsmodels.api as sm
import pandas as pd
from sklearn.preprocessing import scale
from pandas import DataFrame
from sklearn.preprocessing import StandardScaler

plt.rcParams['figure.dpi'] = 300
%config InlineBackend.figure_format = 'svg'

/Users/minyoungxi/.local/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning:

The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.8.0-CAPI-1.13.1). Conversions between both will be slow.



In [24]:
df = pd.read_csv("Korea_income_welfare.csv")
df

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
0,10101,2005,1,1,614.0,1,2,1936,2,2,2,,,8
1,10101,2011,7,1,896.0,1,2,1936,2,2,2,,,10
2,10101,2012,8,1,1310.0,1,2,1936,2,2,2,,,10
3,10101,2013,9,1,2208.0,1,2,1936,2,2,2,,,1
4,10101,2014,10,1,864.0,1,2,1936,2,2,2,,,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92852,98000701,2014,10,5,11600.0,6,1,1967,5,1,1,874,1,
92853,98000701,2015,11,5,8327.0,6,1,1967,5,1,1,874,1,
92854,98000701,2016,12,5,7931.0,6,1,1967,5,1,1,874,1,
92855,98000701,2017,13,5,8802.0,5,1,1967,5,1,1,874,1,


In [12]:
df=df.drop(df[df.income <0].index)

#region (codificamos correctamente todos los datos)
df.loc[df['region'] == 1, 'region'] = 'Seoul'
df.loc[df['region'] == 2, 'region'] = 'Kyeong-gi'
df.loc[df['region'] == 3, 'region'] = 'Kyoung-nam'
df.loc[df['region'] == 4, 'region'] = 'Kyong-buk'
df.loc[df['region'] == 5, 'region'] = 'Chong-nam'
df.loc[df['region'] == 6, 'region'] = 'Gang-won'
df.loc[df['region'] == 7, 'region'] = 'Jeju'

#gender
df.loc[df['gender'] == 1, 'gender'] = 'male'
df.loc[df['gender'] == 2, 'gender'] = 'female'

#religion
df.loc[df['religion'] == 1, 'religion'] = 'religious'
df.loc[df['religion'] == 2, 'religion'] = 'non-religious'

df.loc[df['family_member'] > 7, 'family_member'] = 7


# Calculate natural logarithm on
# 'Salary' column
df['log_income'] = np.log(df['income'])

#eliminamos algunos valores con valores inf, -inf
df=df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

scale= StandardScaler()

df['standarized income']=scale.fit_transform(df[['income']]) 

df1=df.copy()
df1_males = df[df['gender']=='male']
df1_females = df[df['gender']=='female']
df1_males=df1_males.groupby(['year']).mean()
df1_males['year']=df1_males.index
df1_females=df1_females.groupby(['year']).mean()

df1_females['year']=df1_females.index
df2=pd.concat([df1_females, df1_males])
df1_males['income females']=df1_females['income']
df1_males['income males']=df1_males['income']
df1_males.year=df1_males.year.astype(int)
df=df[df['year']==2018]
df.reset_index(inplace=True)
df1_males['ratio females']=(df1_males['income females']/df1_males['income males']).round(2)

In [25]:
df

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
0,10101,2005,1,1,614.0,1,2,1936,2,2,2,,,8
1,10101,2011,7,1,896.0,1,2,1936,2,2,2,,,10
2,10101,2012,8,1,1310.0,1,2,1936,2,2,2,,,10
3,10101,2013,9,1,2208.0,1,2,1936,2,2,2,,,1
4,10101,2014,10,1,864.0,1,2,1936,2,2,2,,,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92852,98000701,2014,10,5,11600.0,6,1,1967,5,1,1,874,1,
92853,98000701,2015,11,5,8327.0,6,1,1967,5,1,1,874,1,
92854,98000701,2016,12,5,7931.0,6,1,1967,5,1,1,874,1,
92855,98000701,2017,13,5,8802.0,5,1,1967,5,1,1,874,1,


In [32]:
df1_males

,id,wave,income,family_member,year_born,education_level,marriage,log_income,standarized income,year,income females,income males,ratio females
year,,,,,,,,,,,,,
2005,3.510980e+07,1.0,2756.578127,3.009797,1953.646948,4.887151,1.360776,7.578049,-0.172458,2005,1090.355862,2756.578127,0.40
2006,3.558404e+07,2.0,3194.647115,3.008386,1953.705666,4.873798,1.378196,7.745685,-0.065103,2006,1244.054575,3194.647115,0.39
2007,3.578503e+07,3.0,3506.341199,2.989467,1953.901548,4.867154,1.384996,7.826672,0.011281,2007,1357.602558,3506.341199,0.39
2008,3.597257e+07,4.0,3700.392286,2.986114,1954.496143,4.891558,1.383513,7.912856,0.058835,2008,1494.715644,3700.392286,0.40
2009,3.606798e+07,5.0,3956.530667,2.970129,1955.044807,4.926700,1.391314,7.981823,0.121605,2009,1610.466408,3956.530667,0.41
2010,3.625001e+07,6.0,4161.694692,2.954155,1955.468252,4.954155,1.387105,8.042094,0.171883,2010,1648.923836,4161.694692,0.40
2011,4.825323e+07,7.0,4068.344785,2.878496,1955.324452,4.924792,1.378496,8.025678,0.149007,2011,1576.135592,4068.344785,0.39
2012,4.783791e+07,8.0,4339.856224,2.880612,1955.734758,4.952362,1.379533,8.090359,0.215544,2012,1637.072611,4339.856224,0.38
2013,4.752637e+07,9.0,4495.345609,2.865899,1956.144591,4.983340,1.373920,8.097349,0.253648,2013,1653.130281,4495.345609,0.37


* Income - Dispersed between 468,209 Won and -232,174 Won, with the majority of income being below 5,000.

* Marriage - There are values that are noted as 0 and 9 which do not have a description associated with it.

* Religion - There should be binary (1 or 2). Currently there is a 9 in the column.

* Occupation, Company_size, Reason_none_worker - Did not show up in the aggregated view. Based on this, it seems like there are some none numeric values in the column.

In [22]:
fig1 = px.bar(df1_males,x = 'year', 
              y = ['income females','income males'],
              range_y=[0,10000],
              range_x=[2004,2019]
            ,height=400,
              template='ggplot2')

fig1.update_layout(paper_bgcolor='rgb(243, 243, 243)',plot_bgcolor='rgb(243, 243, 243)'
,title_text='Average income in million KRW males and females (South Korea 2005 - 2018)',
    font=dict(
        family="Century, monospace",
        color="black"
    ))

fig1.show()